# Day 1

## Part 1

At first I tried to use `map read` on the lines of the input, but that generates a runtime error. Positive frequencies are encoded with a leading `+` sign and Haskell refuses to parse those as one might expect. So here, we define a custom `parseInt` function that pulls off the leading `+` signs before passing the rest onto `read`.

In [1]:
parseInt :: String -> Int
parseInt ('+':rest) = read rest
parseInt str = read str

With that out of the way, we compute the resulting frequency after passing through all of the additions and subtracts by just using the `sum` function on the parsed input.

In [2]:
resultingFrequency :: String -> Int
resultingFrequency = sum . map parseInt . lines

In [3]:
readFile "./inputs/1.txt" >>= print . resultingFrequency

500

## Part 2

In [4]:
import qualified Data.Set as Set

intermediateFrequencies :: [Int] -> [Int]
intermediateFrequencies xs = 0 : f (0, xs)
  where
    f (currentFreq, []) = []
    f (currentFreq, (x:xs)) = let newFreq = currentFreq + x in newFreq : f (newFreq, xs)

firstDuplicate :: (Ord a) => [a] -> a
firstDuplicate xs = f Set.empty xs
  where
    f visited (x:xs) = if x `Set.member` visited
                         then x
                         else f (Set.insert x visited) xs

firstDuplicateFrequency :: String -> Int
firstDuplicateFrequency = firstDuplicate . intermediateFrequencies . cycle . map parseInt . lines

In [5]:
readFile "./inputs/1.txt" >>= print . firstDuplicateFrequency

709

# Day 2

## Part 1

There's a function `group` in `Data.List` that groups together subsequences of equal elements in a list.

In [6]:
import Data.List (group)
group "Mississippi"

["M","i","ss","i","ss","i","pp","i"]

If we apply this to a box ID after we sort its letters we'll get something like this.

In [7]:
let boxId = "evsialkqydurohxqpwbcugtjmh"

import Data.List (sort)
group . sort $ boxId

["a","b","c","d","e","g","hh","i","j","k","l","m","o","p","qq","r","s","t","uu","v","w","x","y"]

Now we can just test if there are any elements of this list that contain exactly 2 characters and test if there are any that contain exactly 3 elements.

In [8]:
any (\x -> length x == 2) . group . sort $ boxId

True

In [9]:
any (\x -> length x == 3) . group . sort $ boxId

False

Putting it all together:

In [10]:
checksum :: [String] -> Int
checksum xs = numIdsWithRepeatedChars 2 * numIdsWithRepeatedChars 3
  where
    numIdsWithRepeatedChars n = length . filter (any (\x -> length x == n) . group . sort) $ xs

In [11]:
day2Input <- readFile "./inputs/2.txt"
checksum . lines $ day2Input

6448

## Part 2

Naively, we can test each string against every other string to count the number of differing characters. I want a function that, given a list, returns a list of pairs of elements.

In [12]:
pairs :: [a] -> [(a,a)]
pairs (x:xs) = map (\y -> (x, y)) xs ++ pairs xs
pairs [] = []

For example,

In [13]:
pairs [1,2,3,4]

[(1,2),(1,3),(1,4),(2,3),(2,4),(3,4)]

Now that we can compute all of the pairs on box IDs we need to compare, we a way to calculate if two box IDs differ by exactly one character.

In [14]:
editDistance :: Eq a => [a] -> [a] -> Int
editDistance xs ys = length . filter (\(x, y) -> x /= y) $ zip xs ys

In [15]:
editDistance "abc" "xyz"

3

In [16]:
editDistance "abc" "azc"

1

Now we can find the unique pair of box IDs whose `editDistance` is 1. But after that, we'll need to construct the string composed of the letters shared between the two.

In [17]:
sharedSubsequence :: Eq a => [a] -> [a] -> [a]
sharedSubsequence xs ys = map fst . filter (\(x, y) -> x == y) $ zip xs ys

In [18]:
sharedSubsequence "abcde" "abzde"

"abde"

Finally,

In [19]:
import Data.List (find)

commonLettersBetweenCorrectBoxIds :: [String] -> Maybe String
commonLettersBetweenCorrectBoxIds xs = uncurry sharedSubsequence <$> pair
  where
    pair = find ((== 1) . uncurry editDistance) . pairs $ xs

In [20]:
commonLettersBetweenCorrectBoxIds . lines $ day2Input

Just "evsialkqyiurohzpwucngttmf"